# Mushroom Classifer Project

In [22]:
import os

In [ ]:
all_images = [x for x in os.listdir('MUSHROOM_images/') if x.endswith('.jpg')]
categories = set([x[:-8] for x in all_images])
category2images = {category : [] for category in categories}
for img in all_images:
    category2images[img[:-8]].append(img)

In [24]:
all_images

['cat_01b.jpg',
 'cat_04s.jpg',
 'dog_01b.jpg',
 'dog_06s.jpg',
 'mango_01b.jpg',
 'mango_03s.jpg']

In [25]:
categories

{'cat', 'dog', 'mango'}

In [26]:
import requests
import torch
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
import numpy as np
from tqdm import tqdm
import pandas as pd
import os

In [27]:
!pip install --upgrade diffusers[torch]
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Athena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Athena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [28]:
from diffusers import StableUnCLIPImg2ImgPipeline
from transformers import CLIPTextModelWithProjection, CLIPTokenizer

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
# ──────────────────────────────────────────────────────────────
# 1.  Load unCLIP – vision side only (projection_dim = 1024)   ─
# ──────────────────────────────────────────────────────────────
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "sd2-community/stable-diffusion-2-1-unclip",
    torch_dtype=torch.float16,
).to(device)

vision_encoder = pipe.image_encoder                       # keep as-is (1024-d)

Loading pipeline components...: 100%|██████████| 9/9 [00:13<00:00,  1.49s/it]
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fai

In [31]:
# ──────────────────────────────────────────────────────────────
# 2.  Swap in an OpenCLIP ViT-H/14 text branch (also 1024-d)  ─
# ──────────────────────────────────────────────────────────────
openclip_repo = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"     # projection_dim = 1024 :contentReference[oaicite:0]{index=0}
tokenizer = CLIPTokenizer.from_pretrained(openclip_repo)
text_encoder = CLIPTextModelWithProjection.from_pretrained(
    openclip_repo,
    torch_dtype=torch.float16
).to(device)

# optional: stuff them into the pipe so `pipe.tokenizer` etc. work
pipe.tokenizer, pipe.text_encoder = tokenizer, text_encoder

In [32]:
# ──────────────────────────────────────────────────────────────
# 3. Helpers                                              
# ──────────────────────────────────────────────────────────────
def embed_images(paths, batch_size=8):
    """Return (N,1024) image embeddings"""
    out, fe, enc = [], pipe.feature_extractor, pipe.image_encoder
    for i in range(0, len(paths), batch_size):
        imgs = [Image.open(p).convert("RGB") for p in paths[i:i + batch_size]]
        px   = fe(imgs, return_tensors="pt").pixel_values.to(enc.device, enc.dtype)
        with torch.no_grad():
            v = enc(px)[0]                              # (B,1024)
        out.append(v)
    return torch.cat(out)  # (N,1024)

def embed_texts(prompts, batch_size=64):
    """Return (N,1024) text embeddings"""
    vecs = []
    for i in range(0, len(prompts), batch_size):
        toks = tokenizer(prompts[i:i + batch_size],
                         padding=True, truncation=True, max_length=77,
                         return_tensors="pt").to(text_encoder.device)
        with torch.no_grad():
            t = text_encoder(**toks).text_embeds        # (B,1024)
        vecs.append(t)
    return torch.cat(vecs)  # (N,1024)
    

In [33]:
os.listdir('THINGS_images/test_images/')

['test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg']

In [ ]:
def classify(pipe, image, label):
    '''
    This function classifies an image using CLIP
    - image: is the image path
    - pipe: is the image encoder + text encoder
    - label: takes in a list of possible labels
    '''

    # labels
    labels = label

    # CLIP image and text
    img = embed_images([image])
    txt = embed_texts(labels)

    #normalize for cosine similarity
    img = torch.nn.functional.normalize(img, dim=-1)
    txt = torch.nn.functional.normalize(txt, dim=-1)

    #compute cosine similarities 
    sim = (img @ txt.T).squeeze(0)

    # pick label
    best = sim.argmax().item()
    best_label = labels[best]
    best_score = sim[best].item()

    return best_label
    #, best_score, sim

In [38]:
print(classify(pipe, "THINGS_images/mango_01b.jpg", ["cat", "mango"]))

mango


# Embed Images

In [ ]:
# embed the images 
# img_embeddings = embed_images(image_path) --> maybe loop through later
# optional: normalize embeddings to unit vectos for cosine similarity here!

# PCA

In [ ]:
# do PCA 
# reduce dimensions to make clustering and visualization easier

# note: this imports PCA, rather than us manually implementing it
from sklearn.decomposition import PCA 

pca = PCA(n_components=50)
X_reduced = pca.fit_transform(img_embeddings.cpu().numpy())

# k-means


In [ ]:
# NOTE: we can use regular image embeddings [img_embeddings] 
# however, we can use the PCA reduced [X_reduced]

# note: this imports KMeans, but we can manually do it as well
from sklearn.cluster import KMeans

# 2 clusters //can change
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_reduced)

# assign each image to a cluster
clusterlabels = kmeans.labels

# Classify tests

In [ ]:
# classifying using text descriptions 
description_labels = ["small brown mushroom with gills", "red mushroom with white spots"]
predictions_desc = [classify(pipe, img, labels=description_labels) for img in all_images]

# classifying using mushroom names 
species_labels = ["placeholder"]
predictions_names = [classify(pipe, img, labels=species_labels) for img in all_images]